# <div align="center"> Machine Learning GradientBoostingRegresor

#### Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt

In [2]:
mercedes = pd.read_csv('mercedes_esp.csv')
audi = pd.read_csv('audi_esp.csv')
land_rover = pd.read_csv('land_rover_esp.csv')
bmw = pd.read_csv('BMW_esp.csv')
porsche = pd.read_csv('porsche_esp.csv')

In [3]:
df = pd.concat([mercedes,audi,land_rover,bmw,porsche],axis = 0)
df=df.reset_index(drop=True)
df.shape

(13094, 7)

In [4]:
limit = len(df)

In [5]:
mercedes_eu = pd.read_csv('mercedes_ue.csv')
audi_eu = pd.read_csv('audi_ue.csv')
land_rover_eu = pd.read_csv('land_rover_eu.csv')
bmw_eu = pd.read_csv('BMW_ue.csv')
porsche_eu = pd.read_csv('porsche_ue.csv')

In [6]:
df = pd.concat([df,mercedes_eu,audi_eu,land_rover_eu,bmw_eu,porsche_eu],axis = 0)
df=df.reset_index(drop=True)
df.shape

(17572, 7)

#### Encoding variables

In [7]:
df_t = pd.get_dummies(df, columns=['Brand','Model','Fuel_type'])

In [8]:
X =  df_t.drop(columns='Price')
y = df_t.loc[:,'Price']

In [9]:
X_eu = X[limit:]
y_eu = y[limit:]
X = X[:limit]
y = y[:limit]

### Train Test Split

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/10, random_state=1)

In [11]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((11784, 96), (11784,), (1310, 96), (1310,))

### GradientBoostingRegressor

In [12]:
from sklearn.ensemble import GradientBoostingRegressor

In [13]:
gbr = GradientBoostingRegressor(loss='huber',criterion='mse',max_depth=8,n_estimators=115)

In [14]:
gbr.fit(X_train,y_train)

GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0, criterion='mse', init=None,
                          learning_rate=0.1, loss='huber', max_depth=8,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=115,
                          n_iter_no_change=None, presort='deprecated',
                          random_state=None, subsample=1.0, tol=0.0001,
                          validation_fraction=0.1, verbose=0, warm_start=False)

------------------------------------------------------------------------------------------------------------------------------

### Predictions

In [15]:
y_pred = gbr.predict(X_test)
d = {'Price':y_test, 'Predictions':y_pred}
test_predictions = pd.DataFrame(data=d)
test_predictions['Error']=test_predictions['Price']-test_predictions['Predictions']
test_predictions['Brand']=df.loc[test_predictions.index,'Brand']
test_predictions['Model']=df.loc[test_predictions.index,'Model']
test_predictions['Year']=df.loc[test_predictions.index,'Year']
test_predictions['Km']=df.loc[test_predictions.index,'Km']
test_predictions['Power']=df.loc[test_predictions.index,'Power']
test_predictions['Fuel_type']=df.loc[test_predictions.index,'Fuel_type']
test_predictions=test_predictions[['Brand', 'Model', 'Year', 'Km', 'Power','Fuel_type', 'Price','Predictions', 'Error']]

### MEAN ABSOLUTE ERROR

In [16]:
abs(test_predictions['Error']).describe()

count     1310.000000
mean      2689.523067
std       2729.709252
min          0.012916
25%        882.872029
50%       1888.999363
75%       3633.043791
max      21571.726864
Name: Error, dtype: float64

-------------------------------------------------------------------------------------------------------------------------------

# European cars predictions

In [17]:
y_pred_eu = gbr.predict(X_eu)
d = {'Price':y_eu, 'Predictions':y_pred_eu}
EU = pd.DataFrame(data=d)
EU['Predictions'] = EU['Predictions'].astype(np.int64)
EU['Error']=EU['Price']-EU['Predictions']
EU['Brand']=df.loc[EU.index,'Brand']
EU['Model']=df.loc[EU.index,'Model']
EU['Year']=df.loc[EU.index,'Year']
EU['Km']=df.loc[EU.index,'Km']
EU['Power']=df.loc[EU.index,'Power']
EU['Fuel_type']=df.loc[EU.index,'Fuel_type']
EU=EU[['Brand', 'Model', 'Year', 'Km', 'Power','Fuel_type', 'Price','Predictions', 'Error']]

In [18]:
EU.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4478 entries, 13094 to 17571
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Brand        4478 non-null   object
 1   Model        4478 non-null   object
 2   Year         4478 non-null   int64 
 3   Km           4478 non-null   int64 
 4   Power        4478 non-null   int64 
 5   Fuel_type    4478 non-null   object
 6   Price        4478 non-null   int64 
 7   Predictions  4478 non-null   int64 
 8   Error        4478 non-null   int64 
dtypes: int64(6), object(3)
memory usage: 315.0+ KB


## Absolute best import opportunities

In [19]:
EU.sort_values(by='Error',ascending=True).head(10)

,Brand,Model,Year,Km,Power,Fuel_type,Price,Predictions,Error
17383,Porsche,Cayenne,2017,89200,420,Gasolina,29500,63941,-34441
15622,LandRover,RangeRoverSport,2016,33407,551,Gasolina,46463,78569,-32106
15611,LandRover,RangeRoverSport,2016,33932,551,Gasolina,46205,77907,-31702
14318,Audi,Q7,2015,142000,272,Diesel,3500,32481,-28981
17566,Porsche,Cayenne,2017,39193,441,Gasolina,58960,85877,-26917
14696,Audi,SQ7,2018,83000,435,Diesel,55000,80702,-25702
15602,LandRover,RangeRoverVelar,2018,32963,300,Diesel,40244,65772,-25528
15562,LandRover,RangeRoverVelar,2018,32995,300,Diesel,40557,65772,-25215
15698,LandRover,DiscoverySport,2018,22000,241,Gasolina,25000,49483,-24483
17456,Porsche,Macan,2017,57800,258,Diesel,28890,53184,-24294


In [20]:
EU.sort_values(by='Error',ascending=True).head(50).to_csv ('Absolute_opportunities.csv', index = False, header=True,sep=';')

##  Best import opportunities [25K - 30K] 

In [21]:
EU[(EU['Price']>25000) &(EU['Price']<30000)].sort_values(by='Error',ascending=True).head(10)

,Brand,Model,Year,Km,Power,Fuel_type,Price,Predictions,Error
17383,Porsche,Cayenne,2017,89200,420,Gasolina,29500,63941,-34441
17456,Porsche,Macan,2017,57800,258,Diesel,28890,53184,-24294
17403,Porsche,Macan,2016,67200,340,Gasolina,29500,53406,-23906
16451,BMW,Serie4,2015,99500,404,Diesel,28000,48580,-20580
13664,Mercedes-Benz,E,2014,100000,408,Gasolina,28300,43212,-14912
17070,Porsche,Cayman,2012,50000,265,Gasolina,29499,42864,-13365
14704,Audi,S1,2017,13400,231,Gasolina,26500,39735,-13235
13684,Mercedes-Benz,GLC,2017,105963,204,Diesel,25990,38617,-12627
16654,BMW,Serie2,2018,57723,340,Gasolina,29900,42111,-12211
17162,Porsche,Cayenne,2014,129000,245,Diesel,28500,39538,-11038


In [22]:
EU[(EU['Price']>25000) &(EU['Price']<30000)].sort_values(by='Error',ascending=True).head(25).to_csv ('Opportunities_25k_30K.csv', index = False, header=True,sep=';')

##  Best import opportunities [20K - 25K] 

In [23]:
EU[(EU['Price']>20000) &(EU['Price']<25000)].sort_values(by='Error',ascending=True).head(10)

,Brand,Model,Year,Km,Power,Fuel_type,Price,Predictions,Error
13890,Mercedes-Benz,GLC,2017,41000,211,Gasolina,21500,41766,-20266
15372,LandRover,RangeRover,2013,144000,249,Diesel,23500,43472,-19972
15801,LandRover,DiscoverySport,2018,67894,179,Diesel,23990,40778,-16788
15405,LandRover,RangeRoverSport,2014,57287,258,Diesel,24000,38986,-14986
15462,LandRover,Discovery,2015,179000,340,Gasolina,22500,36484,-13984
15208,LandRover,RangeRoverSport,2011,111790,510,Gasolina,21500,35136,-13636
17102,Porsche,Cayenne,2012,203874,420,Gasolina,22290,34887,-12597
16923,Porsche,Cayenne,2011,148000,400,Gasolina,23850,34883,-11033
16953,Porsche,Cayenne,2011,97000,290,Gasolina,21999,32723,-10724
15086,LandRover,RangeRover,2011,110000,313,Diesel,22900,33373,-10473


In [24]:
EU[(EU['Price']>20000) &(EU['Price']<25000)].sort_values(by='Error',ascending=True).head(25).to_csv ('Opportunities_20k_25K.csv', index = False, header=True,sep=';')

##  Best import opportunities < 20K

In [25]:
EU[(EU['Price']<20000)].sort_values(by='Error',ascending=True).head(10)

,Brand,Model,Year,Km,Power,Fuel_type,Price,Predictions,Error
14318,Audi,Q7,2015,142000,272,Diesel,3500,32481,-28981
16227,BMW,Serie1,2015,68914,320,Gasolina,10200,29909,-19709
16919,Porsche,Cayenne,2011,135000,400,Gasolina,16900,36534,-19634
17109,Porsche,Boxster,2013,89211,265,Gasolina,18000,36406,-18406
15114,LandRover,RangeRoverSport,2011,114000,510,Gasolina,17000,35136,-18136
13776,Mercedes-Benz,GL,2017,229000,250,Gasolina,11000,28886,-17886
13677,Mercedes-Benz,C,2016,27000,136,Diesel,7999,23946,-15947
14056,Audi,A7,2012,151000,299,Gasolina,10300,25869,-15569
16381,BMW,X5,2015,145399,258,Diesel,13576,28275,-14699
15166,LandRover,RangeRover,2012,300000,313,Diesel,8500,22041,-13541


In [27]:
EU[(EU['Price']<20000)].sort_values(by='Error',ascending=True).head(40).to_csv ('Opportunities_under_20k.csv', index = False, header=True,sep=';')